# Experimet Tracking with Tensorboard

In [1]:
from IPython.display import clear_output
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import efficientnet_b1, EfficientNet_B1_Weights
import matplotlib.pyplot as plt
from pathlib import Path
import os
from torch.utils.tensorboard import SummaryWriter
import numpy as np

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Data Acquisition

In [3]:
BASE_DIR = Path("./pizza_steak_sushi")
TRAIN_DIR = BASE_DIR / "train"
TEST_DIR = BASE_DIR / "test"

In [4]:
if not BASE_DIR.exists():
    !wget https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/data/pizza_steak_sushi.zip -O {BASE_DIR}.zip
    !unzip pizza_steak_sushi.zip -d {BASE_DIR}
    clear_output()

# Weights

In [5]:
weights = EfficientNet_B1_Weights.DEFAULT
weights

EfficientNet_B1_Weights.IMAGENET1K_V2

# Dataset & DataLoader

In [6]:
weights.transforms()

ImageClassification(
    crop_size=[240]
    resize_size=[255]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [7]:
train_transforms = weights.transforms()
test_transforms = weights.transforms()

In [8]:
train = datasets.ImageFolder(root=str(TRAIN_DIR), transform=train_transforms)
test = datasets.ImageFolder(root=str(TEST_DIR), transform=test_transforms)

In [9]:
train.class_to_idx

{'pizza': 0, 'steak': 1, 'sushi': 2}

In [10]:
BATCH_SIZE = 32

train_dl = DataLoader(
    dataset=train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=os.cpu_count() or 1
)

test_dl = DataLoader(
    dataset=test,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=os.cpu_count() or 1
)

# Modelling

## Model

In [11]:
def get_efficientnet_b1():
    model = efficientnet_b1(weights=weights, progress=False).to(device)

    for param in model.features.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features=1280, out_features=len(train.classes))
    ).to(device)

    return model

## Training

In [12]:
def fit(model: nn.Module, dl: DataLoader, val_dl: DataLoader, optim: torch.optim.Optimizer, criterion: nn.Module, epochs=2, writer=None):
    train_loss = []
    val_loss = []

    for epoch in range(epochs):
        model.train()

        batch_loss = []
        batch_acc = []

        for i, (x, y) in enumerate(dl):

            x, y = x.to(device), y.to(device)

            out = model(x)
            loss = criterion(out, y)

            batch_loss.append(loss.item())

            # accuracy calculation
            acc = (torch.softmax(out, dim=1).argmax(dim=1) == y).sum() / len(y)
            batch_acc.append(acc.item())

            optim.zero_grad()
            loss.backward()
            optim.step()

        epoch_train_loss = np.mean(np.array(batch_loss))
        epoch_train_accuracy = np.mean(np.array(batch_acc))

        train_loss.append(epoch_train_loss)

        model.eval()
        with torch.inference_mode():

            batch_loss = []
            batch_acc = []

            for i, (x, y) in enumerate(val_dl):

                x, y = x.to(device), y.to(device)

                out = model(x)
                loss = criterion(out, y)

                batch_loss.append(loss.item())

                # accuracy calculation
                acc = (torch.softmax(out, dim=1).argmax(dim=1) == y).sum() / len(y)
                batch_acc.append(acc.item())

        epoch_val_loss = np.mean(np.array(batch_loss))
        epoch_val_accuracy = np.mean(np.array(batch_acc))

        val_loss.append(epoch_val_loss)

        # Experiment Tracking
        if writer:
            writer.add_scalars(
                "Loss",
                {"train": epoch_train_loss, "val": epoch_val_loss},
                global_step=epoch
            )

            writer.add_scalars(
                "Accuracy",
                {"train": epoch_train_accuracy, "val": epoch_val_accuracy},
                global_step=epoch
            )

    if writer:
        writer.close()

    return train_loss, val_loss

## Experiment Tracking

In [13]:
def create_writer(model, epochs, lr, base_dir=".logs"):
    log_dir = Path(base_dir) / model / f"{epochs}_epochs" / f"lr_{lr}"
    return SummaryWriter(log_dir)

In [14]:
epochs_list = [10, 50]
model_name = "efficient_b1"
lr_list = [1e-3, 1e-4]
criterion = nn.CrossEntropyLoss()

for epochs in epochs_list:
    for lr in lr_list:
        print(f"Epochs {epochs} | LR {lr}")
        writer = create_writer(model_name, epochs=epochs, lr=lr)
        model = get_efficientnet_b1()
        optimizer = torch.optim.Adam(lr=lr, params=model.parameters())
        fit(model, train_dl, test_dl, optimizer, criterion, epochs=epochs, writer=writer)


Epochs 10 | LR 0.001
Epochs 10 | LR 0.0001
Epochs 50 | LR 0.001
Epochs 50 | LR 0.0001


# Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./.logs